# SVG

The shape of a `<path>` element is defined by one parameter: `d`. (See more in basic shapes.) The `d` attribute contains a series of commands and parameters used by those commands.

Each of the commands is instantiated (for example, creating a class, naming and locating it) by a specific letter. For instance, let's move to the `x` and `y` coordinates `(10, 10)`. The "Move to" command is called with the letter `M`. When the parser runs into this letter, it knows it needs to move to a point. So, to move to `(10,10)` the command to use would be `M 10 10`. After that, the parser begins reading for the next command.

# Structure

A triangle is made of $3$ vertices $v1, v2, v3$ where $(v1, v2)$ and $(v2, v3)$ are equal segments.

* The big triangle has side ratio $1 : 1 : \phi$, with $\phi := \frac{\sqrt(5) + 1}{2}$.
* The small triangle has side ratio $1 : 1 : \psi$, where $\psi := \frac{\sqrt(5) - 1}{2}$.


In [33]:
import math

PSI = (math.sqrt(5) - 1) / 2
PSI_2 = 1 - PSI

class Triangle:
    def __init__(self, v1, v2, v3):
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3

    @property
    def d(self):
        side_l, side_r = self.v2 - self.v1, self.v3 - self.v2 

        d = "m {}, {} ".format(self.v1[0], self.v1[1])
        d += "l {}, {} ".format(side_l[0], side_l[1])
        d += "l {}, {} ".format(side_r[0], side_r[1])
        d += "l {}, {}z".format(-side_l[0], -side_l[1])

        return d

    @property
    def center(self):
        return (self.v1 + self.v3)/2
        
class Biggy(Triangle):

    def split(self):
        s1 = PSI_2 * self.v1 + PSI * self.v3
        s2 = PSI_2 * self.v1 + PSI * self.v2

        return (Biggy(s1, s2, self.v1),
                Tiny(s2, s1, self.v2),
                Biggy(self.v3, s1, self.v2))

class Tiny(Triangle):

    def split(self):
        s1 = PSI * self.v1 + PSI_2 * self.v2

        return (Tiny(s1, self.v3, self.v1),
                Biggy(self.v3, s1, self.v2))

In [34]:
def create(intial_tiling=None, depth=5):
    if intial_tiling is None:
        pass
    
    tiling = intial_tiling

    for i in range(depth):
        elems = []
        for elem in tiling:
            elems.extend(elem.split())
        tiling = elems

    # Remove duplicates based on their romboid center
    selements = sorted(tiling, key=lambda elem: ())
    tiling = [selements[0]]
    for i, element in enumerate(selements[1:], start=1):
        if abs(element.centre() - selements[i-1].centre()) > TOL:
            self.elements.append(element)

    return tiling
    

# SVG Generation

In [46]:

import numpy as np

TILE_COLOR = "green"
DEPTH = 5
BASE_STROKE = 10
STROKE_WIDTH = str(PSI ** DEPTH * SCALE * BASE_STROKE)
STROKE_COLOR = "pink"
SCALE = 100
MARGIN = 10

xmin = ymin = - SCALE * MARGIN
width =  height = 2 * SCALE * MARGIN
viewbox ='{} {} {} {}'.format(xmin, ymin, width, height)

svg = ['<?xml version="1.0" encoding="utf-8"?>']   
svg.append('<svg width="100%" height="100%" viewBox="{}"'.format(viewbox))
svg.append(' preserveAspectRatio="xMidYMid meet" version="1.1" baseProfile="full" xmlns="http://www.w3.org/2000/svg">') 

svg.append('<g style="stroke:{}; stroke-width: {};">'.format(STROKE_COLOR, STROKE_WIDTH))

theta = 2 * math.pi / 5
rot = np.array([math.cos(theta), math.sin(theta)])
v1 = np.array([-SCALE/2, 0])
v2 = SCALE/2 * rot
v3 = np.array([SCALE/2/ PSI, 0])

tiling = create(intial_tiling=[Biggy(v1, v2, v3)], depth=DEPTH)

for elem in tiling:
    svg.append('<path fill="{}" opacity="{}" d="{}"/>'.format(TILE_COLOR , 0.3, elem.d))

svg.append('</g>\n</svg>')
svg = '\n'.join(svg)

with open("e1.svg", 'w') as fo:
    fo.write(svg)